In [290]:
import os
import requests
import joblib
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import warnings


In [7]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [291]:
import joblib
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

# Load the weights from the file
weights_file = 'logreg/weights.pkl'
loaded_weights = joblib.load(weights_file)
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words('english'))

In [5]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer

In [4]:
repo_name = "Pushparaj20/t5-small-informal"
file_name = "weights.pkl"
file_url = f"https://huggingface.co/{repo_name}/resolve/main/{file_name}"
output_dir = "logreg"
response = requests.get(file_url)
if response.status_code == 200:
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    # Write the file content to disk
    with open(os.path.join(output_dir, file_name), "wb") as file:
        file.write(response.content)
    print(f"File '{file_name}' downloaded successfully to '{output_dir}'.")
else:
    print(f"Failed to download '{file_name}' from '{repo_name}'. Status code: {response.status_code}")

File 'weights.pkl' downloaded successfully to 'logreg'.


In [ ]:
def replace_with_pos_tags(sentence, weights, is_formal):
      tokens = word_tokenize(sentence)
      stemmed_tokens = [stemmer.stem(token) for token in tokens]  # Stem tokens
      pos_tags = pos_tag(tokens)  # Use original tokens for POS tagging
      replaced_sentence = []

      # Calculate N as described
      N = len(stemmed_tokens) // 4

      # Filter out stop words and stem them
      tokens_without_stopwords = [stemmer.stem(token) for token in tokens if token.lower() not in stop_words]

      # Sort stemmed tokens by weight
      sorted_tokens = sorted(tokens_without_stopwords, key=lambda x: abs(weights.get(x, 0)), reverse=True)

      # Extract N tokens with highest absolute weights
      top_N_tokens = sorted_tokens[:N]

      for token, tag in pos_tags:
          stemmed_token = stemmer.stem(token)
          if stemmed_token in top_N_tokens:
              weight = abs(weights.get(stemmed_token, 0))
              if (is_formal and weight >= 0.01) or (not is_formal and weight >= 0.02):
                  replaced_sentence.append(tag)
              else:
                  replaced_sentence.append(token)
          else:
              replaced_sentence.append(token)

      return ' '.join(replaced_sentence)

In [296]:
def transfer(sentences,id):
  warnings.simplefilter("ignore")
  if id==1:
    model = AutoModelForSeq2SeqLM.from_pretrained("Pushparaj20/t5-base-finetuned")
    tokenizer = AutoTokenizer.from_pretrained("Pushparaj20/t5-base-finetuned")
    is_formal = False
    for sentence in sentences:
      print("Informal: ",sentence)
      t=replace_with_pos_tags(sentence,loaded_weights,is_formal)
      print("Transformed :",t)
      input_ids = tokenizer(t, return_tensors="pt").input_ids
      output = model.generate(input_ids)
      b=output.numpy()
      new=tokenizer.decode(b[0], skip_special_tokens=True)
      print("Formal: ",new)
      print("---------------------------------")



  elif id==2:
    model = AutoModelForSeq2SeqLM.from_pretrained("Pushparaj20/t5-small-informal")
    tokenizer = AutoTokenizer.from_pretrained("Pushparaj20/t5-small-informal")
    is_formal = True
    for sentence in sentences:
      print("Formal: ",sentence)
      t=replace_with_pos_tags(sentence,loaded_weights,is_formal)
      print("Transformed :",t)
      input_ids = tokenizer(t, return_tensors="pt").input_ids
      output = model.generate(input_ids)
      b=output.numpy()
      new=tokenizer.decode(b[0], skip_special_tokens=True)
      print("Informal: ",new)
      print("---------------------------------")

  elif id==3:
    model = AutoModelForSeq2SeqLM.from_pretrained("Pushparaj2811/t5-small-act2pas")
    tokenizer = AutoTokenizer.from_pretrained("Pushparaj2811/t5-small-act2pas")
    for sentence in sentences:
      print("Active: ",sentence)
      input_ids = tokenizer(sentence, return_tensors="pt").input_ids
      output = model.generate(input_ids)
      b=output.numpy()
      new=tokenizer.decode(b[0], skip_special_tokens=True)
      print("Passive ",new)
      print("---------------------------------")
  elif id==4:
    model = AutoModelForSeq2SeqLM.from_pretrained("Pushparaj20/t5-small-pas2act")
    tokenizer = AutoTokenizer.from_pretrained("Pushparaj20/t5-small-pas2act")
    for sentence in sentences:
      print("Passive: ",sentence)
      input_ids = tokenizer(sentence, return_tensors="pt").input_ids
      output = model.generate(input_ids)
      b=output.numpy()
      new=tokenizer.decode(b[0], skip_special_tokens=True)
      print("Active ",new)
      print("---------------------------------")







## **Informal to formal**





In [274]:

sentences = [
    "I kinda have a feeling that he has a crush on you.",
    "car is fucking good.",
    "I am damn worried about my job.",
    "That movie was fucking good",
    "That was funny LOL",
    "It's piece of cake, we can do it",
    "Dude, this car's dope!",
    "OMG! It's finger-lickin' good."
]
transfer(sentences,1)

Informal:  I kinda have a feeling that he has a crush on you.
Transformed : I VBP have a NN that he has a NN on you .
Formal:  I do have a feeling that he has a crush on you.
-----------------------------
Informal:  car is fucking good.
Transformed : car is VBG good .
Formal:  Car is looking good.
-----------------------------
Informal:  I am damn worried about my job.
Transformed : I am RB JJ about my job .
Formal:  I am very concerned about my job.
-----------------------------
Informal:  That movie was fucking good
Transformed : That movie was VBG good
Formal:  That movie was really good.
-----------------------------
Informal:  That was funny LOL
Transformed : That was funny NNP
Formal:  That was funny.
-----------------------------
Informal:  It's piece of cake, we can do it
Transformed : It 's NN of NN , we can do it
Formal:  It's a matter of fact, we can do it.
-----------------------------
Informal:  Dude, this car's dope!
Transformed : NNP , this car 's dope !
Formal:  Yes, th

## **Formal to informal**






In [278]:
sentences = [
    "Please don't leave the room now.",
    "It is a delicious icecream",
    "He is a very nice man and has a charming personality",
    "He is nice person.",
    "think they are good.",
    "It could be true",
    "the band is quite nice.",
    "I am not paying this money to that people",
    "My teacher is good"
]
transfer(sentences,2)

Formal:  Please don't leave the room now.
Transformed : NNP do n't VB the room now .


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Informal:  Don't leave the room now.
---------------------------------
Formal:  It is a delicious icecream
Transformed : It is a JJ icecream
Informal:  It is a good icecream
---------------------------------
Formal:  He is a very nice man and has a charming personality
Transformed : He is a very nice NN and has a charming NN
Informal:  He is a very nice guy and has a charming personality
---------------------------------
Formal:  He is nice person.
Transformed : He is nice NN .
Informal:  He is nice guy.
---------------------------------
Formal:  think they are good.
Transformed : NN they are good .
Informal:  yeah they are good.
---------------------------------
Formal:  It could be true
Transformed : It MD be true
Informal:  It would be true
---------------------------------
Formal:  the band is quite nice.
Transformed : the band is RB nice .
Informal:  the band is really nice.
---------------------------------
Formal:  I am not paying this money to that people
Transformed : I am not

## **Active to Passive**






In [293]:
sentences = [
    "The cat chased the mouse.",
    "She sang a beautiful song.",
    "He built a sandcastle on the beach.",
    "They planted flowers in the garden.",
    "The chef cooked a delicious meal.",
    "The dog fetched the ball.",
    "She painted a colorful picture.",
    "He fixed the broken bicycle.",
    "Did the dog bring back the ball?",
]

transfer(sentences,3)

Active:  The cat chased the mouse.
Passive  The mouse was chased by the cat.
---------------------------------
Active:  She sang a beautiful song.
Passive  A beautiful song was sang by her.
---------------------------------
Active:  He built a sandcastle on the beach.
Passive  A sandcastle was built by him on the beach.
---------------------------------
Active:  They planted flowers in the garden.
Passive  Flowers were planted by them in the garden.
---------------------------------
Active:  The chef cooked a delicious meal.
Passive  A delicious meal was cooked by the chef.
---------------------------------
Active:  The dog fetched the ball.
Passive  The ball was fetched by the dog.
---------------------------------
Active:  She painted a colorful picture.
Passive  A colorful picture was painted by her.
---------------------------------
Active:  He fixed the broken bicycle.
Passive  The broken bicycle was fixed by him.
---------------------------------
Active:  Did the dog bring back t

## **Passive to Active**






In [300]:
sentences = [
    "The cake was baked by Mary.",
    "The book was read by John.",
    "The letter was written by Sarah.",
    "The car was repaired by the mechanic.",
    "The house was built by the construction crew.",
    "The movie was watched by the entire family.",
    "The song was sung by the famous singer.",
    "The problem was solved by the team.",
    "Was the cake baked by Mary?",
    "Was the problem solved by the team?"
]
transfer(sentences,4)


Passive:  The cake was baked by Mary.
Active  Mary baked the cake.
---------------------------------
Passive:  The book was read by John.
Active  John read the book.
---------------------------------
Passive:  The letter was written by Sarah.
Active  Sarah wrote the letter.
---------------------------------
Passive:  The car was repaired by the mechanic.
Active  The mechanic repaired the car.
---------------------------------
Passive:  The house was built by the construction crew.
Active  The construction crew built the house.
---------------------------------
Passive:  The movie was watched by the entire family.
Active  The entire family watched the movie.
---------------------------------
Passive:  The song was sung by the famous singer.
Active  The famous singer sung the song.
---------------------------------
Passive:  The problem was solved by the team.
Active  The team solved the problem.
---------------------------------
Passive:  Was the cake baked by Mary?
Active  Mary baked t